In [2]:
import pandas as pd
train_csv = pd.read_csv('train.csv', delimiter= ",", encoding='utf-8', header=None)
train_csv = train_csv[1:]
train_csv.head()
train_csv[1].head()

1    0
2    4
3    5
4    0
5    4
Name: 1, dtype: object

In [58]:
from matplotlib import pyplot as plt
from collections import Counter
Counter(train_csv[1])
count_0 = 0
count_1 = 0
count_2 = 0
count_3 = 0
count_4 = 0
count_5 = 0
train_final = []
val_final = []
label_size = 200
train_csv.shape
for i in range(17034):
    if(train_csv.iloc[i][1] == '0' and count_0<label_size*1.1):
        count_0+=1
        if(count_0<label_size):
            train_final.append([train_csv.iloc[i][0], train_csv.iloc[i][1]])
        else:
            val_final.append([train_csv.iloc[i][0], train_csv.iloc[i][1]])
    if(train_csv.iloc[i][1] == '1' and count_1<label_size*1.1):
        count_1+=1
        if(count_1<label_size):
            train_final.append([train_csv.iloc[i][0], train_csv.iloc[i][1]])
        else:
            val_final.append([train_csv.iloc[i][0], train_csv.iloc[i][1]])
    if(train_csv.iloc[i][1] == '2' and count_2<label_size*1.1):
        count_2+=1
        if(count_2<label_size):
            train_final.append([train_csv.iloc[i][0], train_csv.iloc[i][1]])
        else:
            val_final.append([train_csv.iloc[i][0], train_csv.iloc[i][1]])
    if(train_csv.iloc[i][1] == '3' and count_3<label_size*1.1):
        count_3+=1
        if(count_3<label_size):
            train_final.append([train_csv.iloc[i][0], train_csv.iloc[i][1]])
        else:
            val_final.append([train_csv.iloc[i][0], train_csv.iloc[i][1]])
    if(train_csv.iloc[i][1] == '4' and count_4<label_size*1.1):
        count_4+=1
        if(count_4<label_size):
            train_final.append([train_csv.iloc[i][0], train_csv.iloc[i][1]])
        else:
            val_final.append([train_csv.iloc[i][0], train_csv.iloc[i][1]])
    if(train_csv.iloc[i][1] == '5' and count_5<label_size*1.1):
        count_5+=1
        if(count_5<label_size):
            train_final.append([train_csv.iloc[i][0], train_csv.iloc[i][1]])
        else:
            val_final.append([train_csv.iloc[i][0], train_csv.iloc[i][1]])

In [59]:
import numpy as np
train_final = np.array(train_final)
val_final = np.array(val_final)
train_final[:,1]
Counter(train_final[:,1])
import cv2
img = cv2.imread('data/data/0.jpg', 0)
img = cv2.resize(img, (150, 150))
(thresh, im_bw) = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

In [62]:
train_final.shape

(1194, 2)

In [97]:
train_x = []
train_y = []
val_x = []
val_y = []
for i in range(1194):
    img_file = train_final[i][0]
    label = train_final[i][1]
    img = cv2.imread('data/data/' + img_file)
    img = cv2.resize(img, (150, 150))
    train_x.append(img.flatten())
    train_y.append(int(label))
    
for i in range(132):
    img_file = val_final[i][0]
    label = val_final[i][1]
    img = cv2.imread('data/data/' + img_file)
    img = cv2.resize(img, (150, 150))
    val_x.append(img.flatten())
    val_y.append(int(label))

In [28]:
test_csv = pd.read_csv('test_WyRytb0.csv', delimiter= ",", encoding='utf-8', header=None)

In [29]:
test_csv = test_csv[1:]

In [32]:
test_csv = np.array(test_csv)

In [35]:
test_csv[1][0]

(7301, 1)

In [98]:
test_x = []
for i in range(7301):
    img_file = test_csv[i][0]
    img = cv2.imread('data/data/' + img_file)
    img = cv2.resize(img, (150, 150))
    im_bw = np.array(img)
    test_x.append(im_bw.flatten())

In [38]:
# Keras imports
from keras.models import  Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
from keras.layers.convolutional import Conv2D, MaxPooling2D

from keras.utils import np_utils

# InceptionV3 model imports
from keras.applications.inception_v3 import InceptionV3

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [39]:
# Hyper parameters
epochs = 1
batch_size = 50

In [42]:

from keras.models import Model
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)
#base_model = VGG16(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(128, activation='relu')(x)
# and a logistic layer -- let's say we have 10 classes
predictions = Dense(6, activation='softmax')(x)

# this is the model we will train
m = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [46]:
for layer in m.layers[:249]:
   layer.trainable = False
for layer in m.layers[249:]:
   layer.trainable = True
   
m.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
print(m.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_189 (Conv2D)             (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_189 (BatchN (None, None, None, 3 96          conv2d_189[0][0]                 
__________________________________________________________________________________________________
activation_189 (Activation)     (None, None, None, 3 0           batch_normalization_189[0][0]    
__________________________________________________________________________________________________
conv2d_190

In [99]:
train_x_array = np.array(train_x)

In [100]:
train_x_array.shape

(1194, 67500)

In [101]:
train_x_pd_reshaped = train_x_array.reshape(train_x_array.shape[0], 150 , 150, 3)

In [102]:
val_x_array = np.array(val_x)
val_x_pd_reshaped = val_x_array.reshape(val_x_array.shape[0], 150 , 150, 3)

In [115]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(sparse=False)
# integer_encoded = np.array(train_y).reshape(len(train_y), 1)
# train_y = onehot_encoder.fit_transform(integer_encoded)
integer_encoded = np.array(val_y).reshape(len(val_y), 1)
val_y = onehot_encoder.fit_transform(integer_encoded)

In [121]:
test_x_array = np.array(test_x)
test_x_pd_reshaped = test_x_array.reshape(test_x_array.shape[0], 150 , 150, 3)

In [109]:
onehot_encoded

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

In [200]:
model_info = m.fit(train_x_pd_reshaped, train_y, validation_data=(val_x_pd_reshaped, val_y), epochs=30, batch_size=batch_size, verbose=2)


Train on 1194 samples, validate on 132 samples
Epoch 1/30
 - 87s - loss: 0.0096 - acc: 0.9983 - val_loss: 1.6560 - val_acc: 0.7879
Epoch 2/30
 - 81s - loss: 0.0189 - acc: 0.9950 - val_loss: 1.0784 - val_acc: 0.7879
Epoch 3/30
 - 80s - loss: 0.0124 - acc: 0.9975 - val_loss: 1.1834 - val_acc: 0.8030
Epoch 4/30
 - 78s - loss: 0.0104 - acc: 0.9983 - val_loss: 1.1136 - val_acc: 0.8030
Epoch 5/30
 - 78s - loss: 0.0168 - acc: 0.9958 - val_loss: 0.9414 - val_acc: 0.8182
Epoch 6/30
 - 78s - loss: 0.0040 - acc: 0.9992 - val_loss: 1.1864 - val_acc: 0.8182
Epoch 7/30
 - 80s - loss: 0.0058 - acc: 0.9983 - val_loss: 1.1950 - val_acc: 0.8182
Epoch 8/30
 - 79s - loss: 0.0028 - acc: 0.9983 - val_loss: 1.2593 - val_acc: 0.8030
Epoch 9/30
 - 78s - loss: 0.0044 - acc: 0.9992 - val_loss: 1.2661 - val_acc: 0.8106
Epoch 10/30
 - 82s - loss: 0.0047 - acc: 0.9975 - val_loss: 1.2600 - val_acc: 0.7955
Epoch 11/30
 - 81s - loss: 0.0068 - acc: 0.9958 - val_loss: 1.3079 - val_acc: 0.7879
Epoch 12/30
 - 78s - loss: 

In [201]:
targets = m.predict(test_x_pd_reshaped, batch_size=32, verbose=0)

In [202]:
targets_final = []
for i in range(7301):
    targets_final.append(np.argmax(targets[i]))

In [203]:
targets_final = np.array(targets_final)

In [204]:
targets_final = np.transpose(targets_final)

In [205]:
f = open("output.csv", "w") 
f.write("image_name,label\n")
for i in range(7301):
    row = str(test_csv[i][0]) + "," + str(targets_final[i]) + "\n"
    f.write(row)
f.close()